<a href="https://colab.research.google.com/github/matheus-giordani/book-suggestions/blob/main/recomenda%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle --upgrade

In [2]:
!pip install joblib

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"matheusgiordani","key":"df1967264bf33ac3632e171ba302dc60"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [5]:
!kaggle datasets download -d rxsraghavagrawal/book-recommender-system

 37% 9.00M/24.5M [00:00<00:00, 38.2MB/s]
100% 24.5M/24.5M [00:00<00:00, 86.5MB/s]


In [6]:
import zipfile
import pandas as pd

In [7]:
files_zip = zipfile.ZipFile('book-recommender-system.zip')
print(files_zip.namelist())

['BX-Book-Ratings.csv', 'BX-Books.csv', 'BX-Users.csv']


In [8]:
books = pd.read_csv(files_zip.open('BX-Books.csv'), sep=';', encoding='latin-1', on_bad_lines='skip')
books = books[['ISBN' ,'Book-Title','Book-Author', 'Year-Of-Publication', 'Publisher']]
books.rename(columns= {'Book-Title': 'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher': 'publisher'}, inplace=True)


<ipython-input-8-c54f343a24b0>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(files_zip.open('BX-Books.csv'), sep=';', encoding='latin-1', on_bad_lines='skip')
<ipython-input-8-c54f343a24b0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books.rename(columns= {'Book-Title': 'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher': 'publisher'}, inplace=True)


In [9]:
users = pd.read_csv(files_zip.open('BX-Users.csv'), sep=';', encoding='latin-1', on_bad_lines='skip')
users.rename(columns = {'User-ID':'user_id', 'Location': 'location', 'Age':'age'}, inplace=True)


In [10]:
ratings = pd.read_csv(files_zip.open('BX-Book-Ratings.csv'), sep=';', encoding='latin-1', on_bad_lines='skip')
ratings.rename(columns = {'User-ID': 'user_id', 'Book-Rating': 'rating'}, inplace=True)

In [11]:
# livros com mais de 200 avaliações
books_more_two_handred_rating = ratings['user_id'].value_counts() > 200
id_users_rate_more_two_handred_rating = books_more_two_handred_rating[books_more_two_handred_rating].index


In [12]:
# traz rating apenas dos usuarios que avaliaram mais de 200 livros
ratings = ratings[ratings['user_id'].isin(id_users_rate_more_two_handred_rating)]
rating_with_books = ratings.merge(books, on='ISBN')

In [13]:
# quantidade de avaliações por livro
number_rating = rating_with_books.groupby('title')['rating'].count().reset_index()
number_rating.rename(columns= { 'rating': 'number_of_ratings'}, inplace=True)



In [14]:
final_rating = rating_with_books.merge(number_rating, on='title')
# Filtrar livros que foram avaliados mais de 50 vezes
final_rating = final_rating[final_rating['number_of_ratings']>= 50]


In [15]:
# elimina usuarios que avaliaram um livro mais de uma vez
final_rating.drop_duplicates(['user_id', 'title'], inplace=True)


<ipython-input-15-1db3abc9454c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(['user_id', 'title'], inplace=True)


In [16]:
final_rating.to_csv('final-rating.csv')

In [17]:
# transpõe os valores de usuario pra que seja uma variável na maquina de predição
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values='rating')


In [18]:
# transforma NaN em 0
book_pivot.fillna(0,inplace=True)

In [19]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Mais sobre matrizes esparsas
> http://www.decom.ufop.br/menotti/aedI082/tps/tp2.pdf

In [20]:
from scipy.sparse import csr_matrix
#  transforma em uma matriz esparsa pra reduzir o custo de processamento
book_sparse = csr_matrix(book_pivot)

In [21]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)


NearestNeighbors(algorithm='brute')

In [22]:
from joblib import dump, load

dump(model, 'recommendation_fit.joblib')



['recommendation_fit.joblib']

In [23]:
# book_pivot.to_csv('book_pivot.csv')
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# 1984
distances, suggestions = model.kneighbors(book_pivot.iloc[0,:].values.reshape(1,-1))
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

Index(['1984', 'No Safe Place', 'A Civil Action', 'Foucault's Pendulum',
       'Long After Midnight'],
      dtype='object', name='title')


In [25]:
# hannibal
distances, suggestions = model.kneighbors(book_pivot.iloc[235,:].values.reshape(1,-1))
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

Index(['Hannibal', 'Vittorio the Vampire: New Tales of the Vampires',
       'The Killing Game: Only One Can Win...and the Loser Dies',
       'The First Wives Club Movie Tie In', 'Exclusive'],
      dtype='object', name='title')
